In [17]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.io import loadmat
from scipy.fftpack import dct
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

In [18]:
def zigzag(matrix):
    zigzag_pattern = [
        [0, 1, 5, 6, 14, 15, 27, 28],
        [2, 4, 7, 13, 16, 26, 29, 42],
        [3, 8, 12, 17, 25, 30, 41, 43],
        [9, 11, 18, 24, 31, 40, 44, 53],
        [10, 19, 23, 32, 39, 45, 52, 54],
        [20, 22, 33, 38, 46, 51, 55, 60],
        [21, 34, 37, 47, 50, 56, 59, 61],
        [35, 36, 48, 49, 57, 58, 62, 63]
    ]

    result = np.zeros(64)
    for i in range(8):
        for j in range(8):
            result[zigzag_pattern[i][j]] = matrix[i, j]
    return result

def dct2(block):
    return dct(dct(block.T, norm='ortho').T, norm='ortho')

def bhattacharyya_distance(mean1, var1, mean2, var2):
    """
    Calculate Bhattacharyya Distance for univariate Gaussian distributions

    Formula for univariate Gaussians:
    DB = 0.25 * ln(0.25 * (σ₁²/σ₂² + σ₂²/σ₁² + 2)) + 0.25 * (μ₁ - μ₂)² * (1/σ₁² + 1/σ₂²)

    Higher distance = better class separation
    Range: [0, ∞)
    """
    var1 = np.where(var1 == 0, 1e-10, var1)
    var2 = np.where(var2 == 0, 1e-10, var2)

    term1 = 0.25 * np.log(0.25 * (var1/var2 + var2/var1 + 2))

    term2 = 0.25 * (mean1 - mean2) ** 2 * (1/var1 + 1/var2)

    return term1 + term2

In [19]:
train_set = scipy.io.loadmat('TrainingSamplesDCT_8_new.mat')

background_data = train_set['TrainsampleDCT_BG']
cheetah_data = train_set['TrainsampleDCT_FG']

c_grass = background_data.shape[0]
c_cheetah = cheetah_data.shape[0]
n = c_grass + c_cheetah

print(f"Number of background samples: {c_grass}")
print(f"Number of cheetah samples: {c_cheetah}")
print(f"Feature dimension: {background_data.shape[1]}")

prior_cheetah = c_cheetah / n
prior_background = c_grass / n

print(f"Prior probability of cheetah: {prior_cheetah:.4f}")
print(f"Prior probability of background: {prior_background:.4f}")

Number of background samples: 1053
Number of cheetah samples: 250
Feature dimension: 64
Prior probability of cheetah: 0.1919
Prior probability of background: 0.8081


In [20]:
print("\n" + "=" * 60)
print("Problem B: Loading training data and computing statistics")
print("=" * 60)

train_set = loadmat('TrainingSamplesDCT_8_new.mat')
FGmat = train_set['TrainsampleDCT_FG']
BGmat = train_set['TrainsampleDCT_BG']

print(f"Foreground samples shape: {FGmat.shape}")
print(f"Background samples shape: {BGmat.shape}")

meanFG = np.mean(FGmat, axis=0)
covFG = np.cov(FGmat, rowvar=False)
varFG = np.var(FGmat, axis=0)

meanBG = np.mean(BGmat, axis=0)
covBG = np.cov(BGmat, rowvar=False)
varBG = np.var(BGmat, axis=0)

mleFG = np.zeros(c_cheetah)
for i in range(c_cheetah):
    tmp = 0
    for j in range(64):
        tmp += ((FGmat[i, j] - meanFG[j]) / np.sqrt(varFG[j])) ** 2
    mleFG[i] = np.exp(-32 * np.log(2 * np.pi) - np.sum(np.log(np.sqrt(varFG))) - 0.5 * tmp)

mleBG = np.zeros(c_grass)
for i in range(c_grass):
    tmp = 0
    for j in range(64):
        tmp += ((BGmat[i, j] - meanBG[j]) / np.sqrt(varBG[j])) ** 2
    mleBG[i] = np.exp(-32 * np.log(2 * np.pi) - np.sum(np.log(np.sqrt(varBG))) - 0.5 * tmp)

print("\nCalculating Bhattacharyya Distance for feature selection...")
bhattacharyya_distances = np.zeros(64)

for i in range(64):
    ave_FG = np.mean(FGmat[:, i])
    variance_FG = np.var(FGmat[:, i])
    ave_BG = np.mean(BGmat[:, i])
    variance_BG = np.var(BGmat[:, i])

    bhattacharyya_distances[i] = bhattacharyya_distance(ave_FG, variance_FG, ave_BG, variance_BG)

print(f"Bhattacharyya distances calculated for all 64 features")
print(f"Bhattacharyya distance range: [{bhattacharyya_distances.min():.4f}, {bhattacharyya_distances.max():.4f}]")

print("\nGenerating marginal plots for all 64 dimensions...")
fig = plt.figure(figsize=(20, 20))

for i in range(64):
    plt.subplot(8, 8, i + 1)

    ave_FG = np.mean(FGmat[:, i])
    variance_FG = np.var(FGmat[:, i])
    sigma_FG = np.sqrt(variance_FG)

    ave_BG = np.mean(BGmat[:, i])
    variance_BG = np.var(BGmat[:, i])
    sigma_BG = np.sqrt(variance_BG)

    xFG = np.linspace(ave_FG - 7 * sigma_FG, ave_FG + 7 * sigma_FG, 350)
    xBG = np.linspace(ave_BG - 7 * sigma_BG, ave_BG + 7 * sigma_BG, 350)
    x = np.sort(np.concatenate([xFG, xBG]))

    y_cheetah = (1 / (sigma_FG * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - ave_FG) / sigma_FG) ** 2)
    y_grass = (1 / (sigma_BG * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - ave_BG) / sigma_BG) ** 2)

    plt.plot(x, y_cheetah, label='Cheetah')
    plt.plot(x, y_grass, label='Grass')
    plt.title(f'Index={i + 1}, B={bhattacharyya_distances[i]:.2f}', fontsize=7)
    plt.tick_params(labelsize=6)

plt.tight_layout()
plt.savefig('64_plots_bhattacharyya.png', dpi=150, bbox_inches='tight')
print("Saved: 64_plots_bhattacharyya.png")
plt.close()


Problem B: Loading training data and computing statistics
Foreground samples shape: (250, 64)
Background samples shape: (1053, 64)

Calculating Bhattacharyya Distance for feature selection...
Bhattacharyya distances calculated for all 64 features
Bhattacharyya distance range: [0.0126, 5.6031]

Generating marginal plots for all 64 dimensions...
Saved: 64_plots_bhattacharyya.png


In [21]:
BhattacharyyaIdx = np.argsort(bhattacharyya_distances)
worstidx = np.sort(BhattacharyyaIdx[:8])
bestidx = np.sort(BhattacharyyaIdx[56:64])

print(f"\nFeature Selection Results (Bhattacharyya Distance):")
print(f"Best 8 features (highest Bhattacharyya distances): {bestidx + 1}")
print(f"  Bhattacharyya distances: {bhattacharyya_distances[bestidx]}")
print(f"Worst 8 features (lowest Bhattacharyya distances): {worstidx + 1}")
print(f"  Bhattacharyya distances: {bhattacharyya_distances[worstidx]}")


Feature Selection Results (Bhattacharyya Distance):
Best 8 features (highest Bhattacharyya distances): [ 1 18 25 27 32 33 40 45]
  Bhattacharyya distances: [5.60314044 0.22422459 0.24552272 0.25195321 0.29200947 0.2462813
 0.23664597 0.22317568]
Worst 8 features (lowest Bhattacharyya distances): [ 3  4  5 59 60 62 63 64]
  Bhattacharyya distances: [0.03646912 0.04215876 0.02369333 0.01717606 0.03547258 0.02522119
 0.01255077 0.01274599]


In [22]:
fig = plt.figure(figsize=(16, 16))

for i in range(8):
    idx_w = worstidx[i]
    idx_b = bestidx[i]

    plt.subplot(4, 4, i + 1)
    ave_FG = np.mean(FGmat[:, idx_b])
    variance_FG = np.var(FGmat[:, idx_b])
    sigma_FG = np.sqrt(variance_FG)
    ave_BG = np.mean(BGmat[:, idx_b])
    variance_BG = np.var(BGmat[:, idx_b])
    sigma_BG = np.sqrt(variance_BG)

    xFG = np.linspace(ave_FG - 7 * sigma_FG, ave_FG + 7 * sigma_FG, 350)
    xBG = np.linspace(ave_BG - 7 * sigma_BG, ave_BG + 7 * sigma_BG, 350)
    x = np.sort(np.concatenate([xFG, xBG]))

    y_cheetah = (1 / (sigma_FG * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - ave_FG) / sigma_FG) ** 2)
    y_grass = (1 / (sigma_BG * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - ave_BG) / sigma_BG) ** 2)

    plt.plot(x, y_cheetah, label='Cheetah')
    plt.plot(x, y_grass, label='Grass')
    plt.title(f'Best {idx_b + 1} (B={bhattacharyya_distances[idx_b]:.2f})', fontsize=10)
    plt.legend(fontsize=8)

    plt.subplot(4, 4, i + 9)
    ave_FG = np.mean(FGmat[:, idx_w])
    variance_FG = np.var(FGmat[:, idx_w])
    sigma_FG = np.sqrt(variance_FG)
    ave_BG = np.mean(BGmat[:, idx_w])
    variance_BG = np.var(BGmat[:, idx_w])
    sigma_BG = np.sqrt(variance_BG)

    xFG = np.linspace(ave_FG - 7 * sigma_FG, ave_FG + 7 * sigma_FG, 350)
    xBG = np.linspace(ave_BG - 7 * sigma_BG, ave_BG + 7 * sigma_BG, 350)
    x = np.sort(np.concatenate([xFG, xBG]))

    y_cheetah = (1 / (sigma_FG * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - ave_FG) / sigma_FG) ** 2)
    y_grass = (1 / (sigma_BG * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - ave_BG) / sigma_BG) ** 2)

    plt.plot(x, y_cheetah, label='Cheetah')
    plt.plot(x, y_grass, label='Grass')
    plt.title(f'Worst {idx_w + 1} (B={bhattacharyya_distances[idx_w]:.2f})', fontsize=10)
    plt.legend(fontsize=8)

plt.tight_layout()
plt.savefig('BestandWorst_plots_bhattacharyya.png', dpi=150, bbox_inches='tight')
print("Saved: BestandWorst_plots_bhattacharyya.png")
plt.close()

Saved: BestandWorst_plots_bhattacharyya.png


In [23]:
print("\n" + "=" * 60)
print("Problem C(i): Classification using 64D Gaussian")
print("=" * 60)

img = np.array(Image.open('cheetah.bmp').convert('L'), dtype=np.float64) / 255.0

img = np.pad(img, ((4, 3), (4, 3)), mode='constant', constant_values=0)

m, n = img.shape
Blocks = np.ones((m - 7, n - 7))

mean_FG_full = meanFG
mean_BG_full = meanBG
inv_covFG = np.linalg.inv(covFG)
inv_covBG = np.linalg.inv(covBG)
DcovFG = np.linalg.det(covFG)
DcovBG = np.linalg.det(covBG)

print("Processing image blocks...")
for i in range(m - 7):
    if (i + 1) % 50 == 0:
        print(f"  Progress: {i + 1}/{m - 7} rows")
    for j in range(n - 7):
        block = img[i:i + 8, j:j + 8]
        DCT = dct2(block)
        feature = zigzag(DCT)
        g_cheetah = (feature @ inv_covFG @ feature.T -
                     2 * feature @ inv_covFG @ mean_FG_full.T +
                     mean_FG_full @ inv_covFG @ mean_FG_full.T +
                     np.log(DcovFG) - 2 * np.log(prior_cheetah))

        g_grass = (feature @ inv_covBG @ feature.T -
                   2 * feature @ inv_covBG @ mean_BG_full.T +
                   mean_BG_full @ inv_covBG @ mean_BG_full.T +
                   np.log(DcovBG) - 2 * np.log(prior_background))

        if g_cheetah >= g_grass:
            Blocks[i, j] = 0

Image.fromarray((Blocks * 255).astype(np.uint8)).save('prediction_64d_bhattacharyya.jpg')
print("Saved: prediction_64d_bhattacharyya.jpg")

ground_truth = np.array(Image.open('cheetah_mask.bmp'), dtype=np.float64) / 255.0
prediction = Blocks

x, y = ground_truth.shape
count1 = 0
count2 = 0
count_cheetah_truth = 0
count_grass_truth = 0

for i in range(x):
    for j in range(y):
        if prediction[i, j] > ground_truth[i, j]:
            count2 += 1
            count_grass_truth += 1
        elif prediction[i, j] < ground_truth[i, j]:
            count1 += 1
            count_cheetah_truth += 1
        elif ground_truth[i, j] > 0:
            count_cheetah_truth += 1
        else:
            count_grass_truth += 1

error1_64 = (count1 / count_cheetah_truth) * prior_cheetah
error2_64 = (count2 / count_grass_truth) * prior_background
total_error_64 = error1_64 + error2_64

print(f"\n64D Results:")
print(f"  Error 1 (miss cheetah): {error1_64:.6f}")
print(f"  Error 2 (false alarm): {error2_64:.6f}")
print(f"  Total error: {total_error_64:.6f}")


Problem C(i): Classification using 64D Gaussian
Processing image blocks...
  Progress: 50/255 rows
  Progress: 100/255 rows
  Progress: 150/255 rows
  Progress: 200/255 rows
  Progress: 250/255 rows
Saved: prediction_64d_bhattacharyya.jpg

64D Results:
  Error 1 (miss cheetah): 0.032885
  Error 2 (false alarm): 0.075293
  Total error: 0.108178


In [24]:
print("\n" + "=" * 60)
print("Problem C(ii): Classification using 8 best features")
print(f"                (Selected by Bhattacharyya Distance)")
print("=" * 60)

img = np.array(Image.open('cheetah.bmp').convert('L'), dtype=np.float64) / 255.0
img = np.pad(img, ((4, 3), (4, 3)), mode='constant', constant_values=0)

m, n = img.shape
Blocks = np.ones((m - 7, n - 7))

mean_FG_best = np.mean(FGmat[:, bestidx], axis=0)
mean_BG_best = np.mean(BGmat[:, bestidx], axis=0)
cov_cheetah = np.cov(FGmat[:, bestidx], rowvar=False)
cov_grass = np.cov(BGmat[:, bestidx], rowvar=False)
inv_covFG_best = np.linalg.inv(cov_cheetah)
inv_covBG_best = np.linalg.inv(cov_grass)
DcovFG_best = np.linalg.det(cov_cheetah)
DcovBG_best = np.linalg.det(cov_grass)

print("Processing image blocks...")
for i in range(m - 7):
    if (i + 1) % 50 == 0:
        print(f"  Progress: {i + 1}/{m - 7} rows")
    for j in range(n - 7):
        block = img[i:i + 8, j:j + 8]
        DCT = dct2(block)
        zigzag_full = zigzag(DCT)
        feature = zigzag_full[bestidx]
        g_cheetah = (feature @ inv_covFG_best @ feature.T -
                     2 * feature @ inv_covFG_best @ mean_FG_best.T +
                     mean_FG_best @ inv_covFG_best @ mean_FG_best.T +
                     np.log(DcovFG_best) - 2 * np.log(prior_cheetah))

        g_grass = (feature @ inv_covBG_best @ feature.T -
                   2 * feature @ inv_covBG_best @ mean_BG_best.T +
                   mean_BG_best @ inv_covBG_best @ mean_BG_best.T +
                   np.log(DcovBG_best) - 2 * np.log(prior_background))

        if g_cheetah >= g_grass:
            Blocks[i, j] = 0

Image.fromarray((Blocks * 255).astype(np.uint8)).save('prediction_8d_bhattacharyya.jpg')
print("Saved: prediction_8d_bhattacharyya.jpg")


Problem C(ii): Classification using 8 best features
                (Selected by Bhattacharyya Distance)
Processing image blocks...
  Progress: 50/255 rows
  Progress: 100/255 rows
  Progress: 150/255 rows
  Progress: 200/255 rows
  Progress: 250/255 rows
Saved: prediction_8d_bhattacharyya.jpg


In [25]:
prediction = Blocks
count1 = 0
count2 = 0
count_cheetah_truth = 0
count_grass_truth = 0

for i in range(x):
    for j in range(y):
        if prediction[i, j] > ground_truth[i, j]:
            count2 += 1
            count_grass_truth += 1
        elif prediction[i, j] < ground_truth[i, j]:
            count1 += 1
            count_cheetah_truth += 1
        elif ground_truth[i, j] > 0:
            count_cheetah_truth += 1
        else:
            count_grass_truth += 1

error1_8 = (count1 / count_cheetah_truth) * prior_cheetah
error2_8 = (count2 / count_grass_truth) * prior_background
total_error_8 = error1_8 + error2_8

print(f"\n8D Results (Bhattacharyya-selected features):")
print(f"  Error 1 (miss cheetah): {error1_8:.6f}")
print(f"  Error 2 (false alarm): {error2_8:.6f}")
print(f"  Total error: {total_error_8:.6f}")

print("\n" + "=" * 60)
print("SUMMARY (Using Bhattacharyya Distance)")
print("=" * 60)
print(f"Feature Selection Method: Bhattacharyya Distance")
print(f"Best 8 features: {bestidx + 1}")
print(f"64D Gaussian - Total Error: {total_error_64:.6f}")
print(f"8D Gaussian  - Total Error: {total_error_8:.6f}")
print("=" * 60)


8D Results (Bhattacharyya-selected features):
  Error 1 (miss cheetah): 0.097131
  Error 2 (false alarm): 0.002353
  Total error: 0.099484

SUMMARY (Using Bhattacharyya Distance)
Feature Selection Method: Bhattacharyya Distance
Best 8 features: [ 1 18 25 27 32 33 40 45]
64D Gaussian - Total Error: 0.108178
8D Gaussian  - Total Error: 0.099484
